In [ ]:
#####  Simulation d'une pompe Cognitive Digital Twin

In [15]:
import pandas as pd
import numpy as np
import requests
import json
import time

# --- 1. Generate Synthetic Pump Sensor Data ---
print("Generating pump sensor data...")
np.random.seed(42)  # For reproducibility

time_steps = 100
time = list(range(time_steps))

# Simulate normal operation for 70 steps, then a fault develops
vibration_normal = np.random.normal(2.0, 0.2, 70)
vibration_fault = np.linspace(2.2, 5.0, 30) + np.random.normal(0, 0.3, 30)
vibration = np.concatenate([vibration_normal, vibration_fault])

# Temperature correlates with vibration
temperature = 50 + (vibration * 5) + np.random.normal(0, 1, time_steps)

# Create DataFrame
df = pd.DataFrame({
    'timestamp': time,
    'vibration_(mm/s)': vibration,
    'temperature_(C)': temperature
})

# Save the "dataset" to a CSV file (your link to the dataset)
dataset_filename = "pump_sensor_data.csv"
df.to_csv(dataset_filename, index=False)
print(f"Synthetic dataset saved as: {dataset_filename}")

# --- 2. Simple Analysis in the "Twin" ---
# Check the last few data points for a rising trend
recent_vibration = df['vibration_(mm/s)'].tail(5)
recent_temperature = df['temperature_(C)'].tail(5)

vibration_trend = np.polyfit(range(5), recent_vibration, 1)[0]  # Slope of the trendline
temp_trend = np.polyfit(range(5), recent_temperature, 1)[0]

is_anomalous = vibration_trend > 0.05 and temp_trend > 0.1 # Arbitrary thresholds

# --- 3. Query the Cognitive Layer (LM Studio) ---
lm_studio_url = "http://localhost:1234/v1/chat/completions"
headers = {"Content-Type": "application/json"}

# Craft a prompt that provides context and data.
prompt = f"""
You are a mechanical engineer analyzing a centrifugal pump.

### SENSOR DATA TREND ###
The last 5 vibration readings (mm/s): {list(recent_vibration.round(2))}.
The last 5 temperature readings (C): {list(recent_temperature.round(2))}.
Analysis: Vibration is showing a {'strong rising' if vibration_trend > 0.05 else 'stable'} trend. Temperature is {'rising' if temp_trend > 0.1 else 'stable'}.

### TASK ###
1.  **Diagnose:** What is the most likely failure mode occurring in the pump based on this data?
2.  **Predict:** If this trend continues, what is the risk in the next 48-72 hours?
3.  **Recommend:** What is your specific maintenance recommendation?

Provide a concise, professional response.
"""

data = {
    "model": "local-model",  # model name isn't strictly needed for LM Studio
    "messages": [
        {"role": "system", "content": "You are an expert mechanical reliability engineer."},
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.2,
    "max_tokens": -1
}

print(prompt)
print(data)

print("\nQuerying Cognitive Digital Twin (via LM Studio)...")
try:
    response = requests.post(lm_studio_url, headers=headers, json=data, timeout=120)
    if response.status_code == 200:
        reply = response.json()['choices'][0]['message']['content']
        print("\n--- COGNITIVE DIGITAL TWIN ANALYSIS ---")
        print(reply)
    else:
        print(f"Error: HTTP {response.status_code}")
        print(response.text)
except Exception as e:
    print(f"Error calling LM Studio: {e}")
    print("Make sure LM Studio local server is running on port 1234.")


Generating pump sensor data...
Synthetic dataset saved as: pump_sensor_data.csv

You are a mechanical engineer analyzing a centrifugal pump.

### SENSOR DATA TREND ###
The last 5 vibration readings (mm/s): [4.17, 4.8, 4.89, 4.9, 4.93].
The last 5 temperature readings (C): [71.26, 73.11, 74.58, 74.58, 73.51].
Analysis: Vibration is showing a strong rising trend. Temperature is rising.

### TASK ###
1.  **Diagnose:** What is the most likely failure mode occurring in the pump based on this data?
2.  **Predict:** If this trend continues, what is the risk in the next 48-72 hours?
3.  **Recommend:** What is your specific maintenance recommendation?

Provide a concise, professional response.

{'model': 'local-model', 'messages': [{'role': 'system', 'content': 'You are an expert mechanical reliability engineer.'}, {'role': 'user', 'content': '\nYou are a mechanical engineer analyzing a centrifugal pump.\n\n### SENSOR DATA TREND ###\nThe last 5 vibration readings (mm/s): [4.17, 4.8, 4.89, 4.9, 

In [ ]:
#### estimation du RUL  remaining useful life (RUL)  d'une turbine 

In [13]:
import requests
import json

# LM Studio server settings
url = "http://localhost:1234/v1/chat/completions"

# Your structured data from the digital twin backend
turbine_data = {
    "Turbine_ID": "T25",
    "Current_RUL": 15,
    "Sensor_Readings": {
        "Sensor_12": {"value": 520.4, "status": "High"},
        "Sensor_7": {"value": 45.2, "status": "Normal"},
        "Sensor_21": {"value": 1100.1, "status": "High"}
    },
    "Top_Risk_Factors": ["Sensor_12 - Overheating Trend", "Sensor_21 - Vibration Spike"]
}

# The prompt crafted by the backend
system_message = f"""You are a helpful and precise Cognitive Digital Twin for a wind turbine. You are given structured data about the turbine's status. Provide clear, concise, and factual answers based ONLY on the following data:

{json.dumps(turbine_data, indent=2)}"""

user_question = "What is the current remaining useful life of turbine T25, and what are the top two sensor contributors to this prediction?"

# The payload for the LM Studio API
payload = {
    "model": "deepseek-coder-v2-lite-instruct", # Should match the model you loaded   mistral-7b-instruct
    "messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_question}
    ],
    "temperature": 0.1, # Low temperature for more factual, deterministic answers
    "max_tokens": 200
}

# Send the request
response = requests.post(url, json=payload)
result = response.json()

# Print the AI's response
print(result['choices'][0]['message']['content'])

 The current remaining useful life (RUL) of turbine T25 is 15 years. The top two sensor contributors to this prediction, based on the provided data, are Sensor_12 showing an overheating trend and Sensor_21 indicating a vibration spike.
